In [ ]:
from sklearn.model_selection import GroupShuffleSplit
import pandas as pd 

In [ ]:
import os
import os.path
from google.colab import drive

# mount Google Drive to /content/drive/My Drive/
if os.path.isdir("/content/drive/MyDrive/Web Mining"):
  print("Google Drive already mounted")
else:
  drive.mount('/content/drive',force_remount=True)  
os.chdir("/content/drive/MyDrive/Web Mining")

Mounted at /content/drive


In [ ]:
users = pd.read_csv("./Abhay/users_grouped.csv", skipinitialspace=True)
gs = GroupShuffleSplit(n_splits=2, test_size=0.2, random_state=42)
train_ix, test_ix = next(gs.split(users, groups=users["Group"]))
# print(users.head())
# print(len(users))
train = users.loc[train_ix]
# print(train) 
test = users.loc[test_ix]
# print(test)
# print(train['Group'].nunique()) # correct we have 80% users in train, i.e. also 80% groups are in train 

In [ ]:
ratings = pd.read_csv("./ratings_new.csv")
ratings = ratings.drop('Timestamp', axis=1) 
print(ratings.head())

   UserID  MovieID  Rating
0       1     1193       5
1       1      661       3
2       1      914       3
3       1     3408       4
4       1     2355       5


In [ ]:
act_ratings = ratings.copy() 
act_ratings = act_ratings.loc[act_ratings['UserID'].isin(train['UserID'])]
act_ratings.reset_index(drop=True, inplace=True) # made sure the users are all in train! 
# print(act_ratings)

In [ ]:
movies = pd.read_csv("./movie_final_full_dataset.csv")

In [ ]:
act_movies = movies.copy() 
act_movies = act_movies.loc[act_movies['MovieID'].isin(act_ratings['MovieID'])]
act_movies = act_movies.drop(columns=["Links", "description", "duration", "ratings", "Title", "ImdbID", "title", "year"])
print(act_movies)

      MovieID                                       Genres  \
0           1  Adventure|Animation|Children|Comedy|Fantasy   
1           2                   Adventure|Children|Fantasy   
2           3                               Comedy|Romance   
3           4                         Comedy|Drama|Romance   
4           5                                       Comedy   
...       ...                                          ...   
3357     3948                                       Comedy   
3358     3949                                        Drama   
3359     3950                                        Drama   
3360     3951                                        Drama   
3361     3952                               Drama|Thriller   

                    director  \
0          ['John Lasseter']   
1           ['Joe Johnston']   
2          ['Howard Deutch']   
3        ['Forest Whitaker']   
4          ['Charles Shyer']   
...                      ...   
3357           ['Jay Roach']   

In [ ]:
# print(act_movies.head())

In [ ]:
import re

for column in act_movies: 
  if (act_movies[column].dtype == object):
      act_movies[column] = act_movies[column].apply(lambda x: x.lower())
      act_movies[column] = act_movies[column].apply(lambda x: re.sub('[^A-Za-z0-9]+', ' ', x))
print(act_movies.head)

<bound method NDFrame.head of       MovieID                                       Genres  \
0           1  adventure animation children comedy fantasy   
1           2                   adventure children fantasy   
2           3                               comedy romance   
3           4                         comedy drama romance   
4           5                                       comedy   
...       ...                                          ...   
3357     3948                                       comedy   
3358     3949                                        drama   
3359     3950                                        drama   
3360     3951                                        drama   
3361     3952                               drama thriller   

                  director                                          writer  \
0           john lasseter        john lasseter pete docter andrew stanton    
1            joe johnston       jonathan hensleigh greg taylor jim st

In [ ]:
!pip install tensorflow-text
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.6 MB 4.4 MB/s 
     |████████████████████████████████| 511.7 MB 6.1 kB/s 
     |████████████████████████████████| 1.6 MB 44.0 MB/s 
     |████████████████████████████████| 438 kB 66.8 MB/s 
     |████████████████████████████████| 5.8 MB 37.2 MB/s 
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.8.0
    Uninstalling tensorflow-estimator-2.8.0:
      Successfully uninstalled tensorflow-estimator-2.8.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.8.0
    Uninstalling tensorboard-2.8.0:
      Successfully uninstalled tensorboard-2.8.0
  Attempting uninstall: keras
    Found existing installation: keras 2.8.0
    Uninstalling keras-2.8.0:
      Successfully uninstalled keras-2.8.0
  Attempting uninstall: gast
    Found existing installation: gast 0.5.3
    Uninst

In [ ]:
preprocessor = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-128_A-2/1", trainable=True)

def get_bert_embeddings(text, preprocessor, encoder):
  text_input = tf.keras.layers.Input(shape=(), dtype=tf.string)
  encoder_inputs = preprocessor(text_input)
  outputs = encoder(encoder_inputs)
  embedding_model = tf.keras.Model(text_input, outputs['pooled_output'])
  sentences = tf.constant([text])
  return embedding_model(sentences)

In [ ]:
act_movies['all'] = ""
for column in act_movies: 
  if (act_movies[column].dtype == object):
    act_movies['all'] = act_movies[['all', column]].apply(" ".join, axis=1)

In [ ]:
print(act_movies.head())

   MovieID                                       Genres           director  \
0        1  adventure animation children comedy fantasy     john lasseter    
1        2                   adventure children fantasy      joe johnston    
2        3                               comedy romance     howard deutch    
3        4                         comedy drama romance   forest whitaker    
4        5                                       comedy     charles shyer    

                                           writer  \
0       john lasseter pete docter andrew stanton    
1      jonathan hensleigh greg taylor jim strain    
2                            mark steven johnson    
3                     terry mcmillan ronald bass    
4   albert hackett frances goodrich nancy meyers    

                                             stars  \
0                 tom hanks tim allen don rickles    
1        robin williams kirsten dunst bonnie hunt    
2          walter matthau jack lemmon ann margret 

In [ ]:
act_movies['encodings'] = act_movies['all'].apply(lambda x: get_bert_embeddings(x, preprocessor, encoder))

In [ ]:
import numpy as np 

saved_act_movies = act_movies.copy()
x = np.vstack(saved_act_movies['encodings'])

In [ ]:
x

array([[-0.984103  , -0.12792562,  0.34944913, ...,  0.99686646,
         0.9735833 ,  0.8820352 ],
       [-0.99390584, -0.06363527,  0.15075861, ...,  0.9915727 ,
         0.8645415 ,  0.77625906],
       [-0.99958396, -0.08141177, -0.2664953 , ...,  0.99598736,
         0.96773005,  0.7681743 ],
       ...,
       [-0.9980589 , -0.11176378, -0.3644842 , ...,  0.9840637 ,
         0.77282184,  0.9084081 ],
       [-0.99954724,  0.02750927, -0.35465938, ...,  0.9705843 ,
         0.8472266 ,  0.8241235 ],
       [-0.9987033 , -0.03651239, -0.1393774 , ...,  0.9981851 ,
         0.83045065,  0.8162273 ]], dtype=float32)

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
cos = cosine_similarity(x)

In [ ]:
cos

array([[0.99999994, 0.9321912 , 0.91337484, ..., 0.8702288 , 0.74985033,
        0.94502115],
       [0.9321912 , 0.99999994, 0.9388789 , ..., 0.9436089 , 0.8257735 ,
        0.9537765 ],
       [0.91337484, 0.9388789 , 1.0000002 , ..., 0.95004284, 0.91261864,
        0.97456056],
       ...,
       [0.8702288 , 0.9436089 , 0.95004284, ..., 0.9999998 , 0.8956819 ,
        0.94619745],
       [0.74985033, 0.8257735 , 0.91261864, ..., 0.8956819 , 0.99999976,
        0.8580114 ],
       [0.94502115, 0.9537765 , 0.97456056, ..., 0.94619745, 0.8580114 ,
        1.        ]], dtype=float32)

In [ ]:
saved_act_movies = movies.copy() 
saved_act_movies = saved_act_movies.loc[saved_act_movies['MovieID'].isin(act_ratings['MovieID'])]
saved_act_movies = saved_act_movies.drop(columns=["Links", "description", "duration", "ratings", "ImdbID", "title", "year"])
print(len(saved_act_movies))

3362


In [ ]:
#Construct a reverse map of indices and movie titles
indices = pd.Series(saved_act_movies.index, index=saved_act_movies['MovieID']).drop_duplicates()
print(indices[:10])
list_act = act_ratings.values.tolist()

MovieID
1     0
2     1
3     2
4     3
5     4
6     5
7     6
8     7
9     8
10    9
dtype: int64


In [ ]:
all_ratings = pd.DataFrame(columns=['UserID', 'MovieID', 'Rating', 'Ranking'])
all_ratings.head()

,UserID,MovieID,Rating,Ranking


In [ ]:
def get_recommendations(user, cosine_sim=cos):
  # Get the index of the movie that matches the title
  list_act_filtered = filter(lambda c: c[0] == user, list_act)
  ratings_watched = (list([x[2] for x in list_act_filtered]))
  list_act_filtered = filter(lambda c: c[0] == user, list_act)
  movies_watched = (list([x[1] for x in list_act_filtered]))
  sim_scores = [[0, 0]] * 3362
  for index, (value1, value2) in enumerate(zip(movies_watched, ratings_watched)): 
    idx = indices[value1]
    test = list(enumerate(cosine_sim[idx]))
    for index2, x in enumerate(test): 
      x = list(x)
      x[1] = x[1] * value2 + sim_scores[index2][1]
      sim_scores[index2] = x 
  len_watched = len(movies_watched)
  for index3, x in enumerate(sim_scores): 
      x = list(x)
      x[1] = x[1] / len_watched
      sim_scores[index3] = x 

  filtered = filter(lambda c: c[0] not in movies_watched, sim_scores)
  sim_scores = list(filtered)

  # Sort the movies based on the similarity scores
  sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

  # Get the scores of the 10 most similar movies
  sim_scores_short = sim_scores[0:10]
  
  # Get the movie indices
  movie_indices = [i[0] for i in sim_scores_short]

  # Return the top 10 most similar movies
  # print(saved_act_movies['Title'].iloc[movie_indices]) 

  # for dataframe
  for index4, x in enumerate(sim_scores): 
    x.append(index4 + 1) 
    x.insert(0, user)

  return(sim_scores)

In [ ]:
# df = pd.DataFrame(get_recommendations(1993),columns=['col1','col2', '3', '4'])
# df.head()
from tqdm import tqdm
final_output = [] 
for user in tqdm(train['UserID']):
  final_output.append(get_recommendations(user))
  # all_ratings.append(df, columns=['UserID', 'MovieID', 'Rating', 'Ranking']))

100%|██████████| 4826/4826 [2:41:39<00:00,  2.01s/it]


In [ ]:
final_output

In [ ]:
final_list = [] 

for layer in final_output: 
  for two in layer: 
    final_list.append(two)

In [ ]:


# all_ratings = pd.DataFrame(final_output, columns = ['UserID', 'MovieID', 'Rating', 'Ranking'])

In [ ]:
import numpy as np
X = np.array(final_list) 
X.shape

(15526977, 4)

In [ ]:
all_ratings = pd.DataFrame(final_list, columns = ['UserID', 'MovieID', 'Rating', 'Ranking'])

In [ ]:
all_ratings 

,UserID,MovieID,Rating,Ranking
0,6040,1830,3.321054,1
1,6040,1463,3.320660,2
2,6040,550,3.317724,3
3,6040,293,3.315502,4
4,6040,75,3.312456,5
...,...,...,...,...
15526972,1,640,2.674799,3306
15526973,1,1185,2.621936,3307
15526974,1,2575,2.606912,3308
15526975,1,1296,2.591984,3309


In [ ]:
all_ratings.to_csv('./Final Data/single_bert_user.csv', index=False)

In [ ]:
test

,UserID,Cluster,Gender,Age,Occupation,Group
20,3949,0,M,45,0,3
21,2629,0,M,25,11,3
22,3921,0,M,25,14,3
23,1538,0,M,25,3,3
24,3926,0,M,25,2,3
...,...,...,...,...,...,...
6004,4965,19,M,1,10,593
6005,2199,19,M,1,10,593
6006,762,19,M,1,10,593
6007,2202,19,F,1,10,593


In [ ]:
act_ratings_test = ratings.copy() 
act_ratings_test = act_ratings_test.loc[act_ratings_test['UserID'].isin(test['UserID'])]
act_ratings_test.reset_index(drop=True, inplace=True) # made sure the users are all in test! 

In [ ]:
print(act_ratings_test)

        UserID  MovieID  Rating
0            2     1357       5
1            2     3068       4
2            2     1537       4
3            2      647       3
4            2     2194       4
...        ...      ...     ...
209641    6038     1387       2
209642    6038     2700       1
209643    6038     2716       3
209644    6038     3396       3
209645    6038     1079       5

[209646 rows x 3 columns]


In [ ]:
#Construct a reverse map of indices and movie titles
saved_act_movies_test = movies.copy() 
saved_act_movies_test = saved_act_movies_test.loc[saved_act_movies_test['MovieID'].isin(act_ratings['MovieID'])]
saved_act_movies_test = saved_act_movies_test.drop(columns=["Links", "description", "duration", "ratings", "ImdbID", "title", "year"])
print(len(saved_act_movies_test))

3362


In [64]:
indices = pd.Series(saved_act_movies_test.index, index=saved_act_movies_test['MovieID']).drop_duplicates()
print(indices[:10])
list_act_test = act_ratings_test.values.tolist()
print(list_act_test[0])

MovieID
1     0
2     1
3     2
4     3
5     4
6     5
7     6
8     7
9     8
10    9
dtype: int64
[2, 1357, 5]


In [ ]:
all_ratings_test = pd.DataFrame(columns=['UserID', 'MovieID', 'Rating', 'Ranking'])
all_ratings_test.head()

,UserID,MovieID,Rating,Ranking


In [82]:
def get_recommendations_test(user, cosine_sim=cos):
  # Get the index of the movie that matches the title
  list_act_filtered = filter(lambda c: c[0] == user, list_act_test)
  ratings_watched = (list([x[2] for x in list_act_filtered]))
  list_act_filtered = filter(lambda c: c[0] == user, list_act_test)
  movies_watched = (list([x[1] for x in list_act_filtered]))
  sim_scores = [[0, 0]] * 3362
  for index, (value1, value2) in enumerate(zip(movies_watched, ratings_watched)): 
    idx = indices[value1]
    test = list(enumerate(cosine_sim[idx]))
    for index2, x in enumerate(test): 
      x = list(x)
      x[1] = x[1] * value2 + sim_scores[index2][1]
      sim_scores[index2] = x 
  len_watched = len(movies_watched)
  for index3, x in enumerate(sim_scores): 
      x = list(x)
      x[1] = x[1] / len_watched
      sim_scores[index3] = x 

  # Sort the movies based on the similarity scores
  sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

  # Get the scores of the 10 most similar movies
  sim_scores_short = sim_scores[0:10]
  
  # Get the movie indices
  movie_indices = [i[0] for i in sim_scores_short]

  # Return the top 10 most similar movies
  # print(saved_act_movies['Title'].iloc[movie_indices]) 

  # for dataframe
  for index4, x in enumerate(sim_scores): 
    x.append(index4 + 1) 
    x.insert(0, user)

  return(sim_scores)

In [91]:
from tqdm import tqdm
final_output_test = [] 
for user in tqdm(test['UserID']):
  final_output_test.append(get_recommendations_test(user)) 

100%|██████████| 1214/1214 [40:35<00:00,  2.01s/it]


In [97]:
final_list_test = [] 

for layer in final_output_test: 
  for two in layer: 
    final_list_test.append(two)

In [98]:
final_list_test[0]

[3949, 75, 3.6210081916716366, 1]

In [99]:
import numpy as np
X = np.array(final_list_test) 
X.shape
# final_list_test[0]

(4081468, 4)

In [100]:
all_ratings_test = pd.DataFrame(final_list_test, columns = ['UserID', 'MovieID', 'Rating', 'Ranking'])

In [101]:
all_ratings_test.to_csv('./Final Data/single_bert_test.csv', index=False)

In [ ]:
get_recommendations(1993)

[[0, 3.4599301159381866], [2, 3.5256978243589403], [3, 3.504677450656891], [4, 3.5427824944257735], [5, 3.351701819896698], [6, 3.542205846309662], [7, 3.417726990580559], [8, 3.403596433997154], [9, 3.435414198040962], [10, 3.4605669498443605], [11, 3.5194041460752485], [12, 3.4994480282068254], [13, 3.4455061554908752], [14, 3.5335154950618746], [15, 3.10689962208271], [16, 3.4005301266908647], [17, 3.5010460376739503], [18, 3.515427830815315], [19, 3.3904217809438704], [20, 3.4774797350168227], [21, 3.500864803791046], [22, 3.378563421964645], [23, 3.2650709718465807], [24, 3.482733318209648], [25, 3.4060331255197527], [26, 3.4059611052274703], [27, 3.4548149436712263], [28, 3.4802245527505873], [29, 2.7611678421497343], [30, 3.463097983598709], [31, 3.510579127073288], [32, 3.4581812798976896], [33, 3.463931608200073], [35, 3.061646154522896], [36, 3.538359534740448], [37, 3.5232127875089647], [38, 3.4830749303102495], [39, 3.1769821524620054], [40, 3.5023591041564943], [41, 3.3362

[[1993, 1264, 3.5810866445302962, 1],
 [1993, 732, 3.5754269868135453, 2],
 [1993, 1870, 3.5710229724645615, 3],
 [1993, 1413, 3.5702902555465696, 4],
 [1993, 2511, 3.5687377572059633, 5],
 [1993, 1874, 3.568484976887703, 6],
 [1993, 2705, 3.5682621896266937, 7],
 [1993, 3055, 3.568048456311226, 8],
 [1993, 2448, 3.568026027083397, 9],
 [1993, 729, 3.567977824807167, 10],
 [1993, 1741, 3.567137894034386, 11],
 [1993, 3026, 3.5670761078596116, 12],
 [1993, 2779, 3.566357523202896, 13],
 [1993, 1727, 3.5663295805454256, 14],
 [1993, 2787, 3.5657100439071656, 15],
 [1993, 2606, 3.5652640134096147, 16],
 [1993, 1962, 3.564638155698776, 17],
 [1993, 3200, 3.564423832297325, 18],
 [1993, 1686, 3.5636717051267626, 19],
 [1993, 1048, 3.563529598712921, 20],
 [1993, 1658, 3.5626958131790163, 21],
 [1993, 1808, 3.5625746876001356, 22],
 [1993, 2263, 3.5625714868307115, 23],
 [1993, 292, 3.5622343868017197, 24],
 [1993, 163, 3.5619839638471604, 25],
 [1993, 866, 3.5618995070457458, 26],
 [1993, 1

In [ ]:
# check what movies he watched
z = act_ratings['MovieID'].loc[act_ratings['UserID'] == 1993]
for x in z: 
  print(saved_act_movies['Title'].loc[saved_act_movies['MovieID'] == x])

1040    Better Off Dead... (1985)
Name: Title, dtype: object
0    Toy Story (1995)
Name: Title, dtype: object
1718    Lady and the Tramp (1955)
Name: Title, dtype: object
1071    This Is Spinal Tap (1984)
Name: Title, dtype: object
1901    Player, The (1992)
Name: Title, dtype: object
204    Clerks (1994)
Name: Title, dtype: object
2942    High Fidelity (2000)
Name: Title, dtype: object
933    Return of the Pink Panther, The (1974)
Name: Title, dtype: object
942    Monty Python and the Holy Grail (1974)
Name: Title, dtype: object
3120    Porky's II: The Next Day (1983)
Name: Title, dtype: object
982    Star Wars: Episode V - The Empire Strikes Back...
Name: Title, dtype: object
983    Princess Bride, The (1987)
Name: Title, dtype: object
984    Raiders of the Lost Ark (1981)
Name: Title, dtype: object
1162    Raising Arizona (1987)
Name: Title, dtype: object
2001    Rushmore (1998)
Name: Title, dtype: object
2002    Shakespeare in Love (1998)
Name: Title, dtype: object
303    Star Trek

Original code

In [ ]:
saved_act_movies = act_movies.copy()
print(saved_act_movies)
act_movies = saved_act_movies 
# act_movies.to_csv("encoded_movies.csv")

# act_movies = pd.read_csv('./encoded_movies.csv')
print(act_movies)

      MovieID                                       Genres  \
0           1  adventure animation children comedy fantasy   
1           2                   adventure children fantasy   
2           3                               comedy romance   
3           4                         comedy drama romance   
4           5                                       comedy   
...       ...                                          ...   
3357     3948                                       comedy   
3358     3949                                        drama   
3359     3950                                        drama   
3360     3951                                        drama   
3361     3952                               drama thriller   

                  director                                          writer  \
0           john lasseter        john lasseter pete docter andrew stanton    
1            joe johnston       jonathan hensleigh greg taylor jim strain    
2           howard de

In [ ]:
import torch
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np 
# rm = np.array(act_movies['encodings'].loc[act_movies['MovieID'] == 9])
# rm = np.vstack(rm).astype(np.float)
# print(cosine_similarity(rm, rm))

# ls = np.vstack(np.array(list_by_user.encodings.iloc[0])).astype(np.float)
# print(cosine_similarity(rm, ls))


In [ ]:
pred_ratings = pd.DataFrame(columns=['UserID', 'MovieID', 'Rating'])
pred_ratings.head()

,UserID,MovieID,Rating


In [ ]:
from statistics import mean

# for user in users['UserID'].unique():
#   list_by_user = act_movies[['MovieID', 'encodings']].loc[act_movies['MovieID'].isin(act_ratings['MovieID'].loc[act_ratings['UserID'] == user.UserID])]
#   print(list_by_user)
# list_by_user.reset_index(drop=True, inplace=True)
# list_by_user['Rating'] = 0

# for movie in list_by_user.itertuples(): 
#   # print(act_ratings['Rating'].loc[(act_ratings['UserID'] == user) & (act_ratings['MovieID'] == movie.MovieID)])
#   # list_by_user.at[0, 'MovieID'] = 1000
#   list_by_user.at[movie.Index, 'Rating'] = act_ratings['Rating'].loc[(act_ratings['UserID'] == user) & (act_ratings['MovieID'] == movie.MovieID)]

# print((list_by_user))
from tqdm import tqdm 

for user in tqdm(users.head(100).itertuples()): 
  list_by_user = act_movies[['MovieID', 'encodings']].loc[act_movies['MovieID'].isin(act_ratings['MovieID'].loc[act_ratings['UserID'] == user.UserID])]
  list_by_user.reset_index(drop=True, inplace=True)
  list_by_user['Rating'] = 0

  for movie in list_by_user.itertuples(): 
    list_by_user.at[movie.Index, 'Rating'] = act_ratings['Rating'].loc[(act_ratings['UserID'] == user.UserID) & (act_ratings['MovieID'] == movie.MovieID)]

  for movie in act_movies.itertuples():
    if ((movie.MovieID in list_by_user.MovieID.values) == False):
      rm = np.array(act_movies['encodings'].loc[act_movies['MovieID'] == movie.MovieID])
      rm = np.vstack(rm).astype(float)

      my_new_list = [] 
      for i in list_by_user.itertuples(): 
        my_new_list.append((cosine_similarity(rm, np.vstack(np.array(i.encodings)).astype(float)) * i.Rating))
      new_rating = np.array(my_new_list).mean()
      pred_ratings.loc[len(pred_ratings.index)] = [user.UserID, movie.MovieID, new_rating]
      
  print("one line")


1it [04:27, 267.65s/it]

one line


2it [07:34, 220.23s/it]

one line


3it [08:30, 145.00s/it]

...

one line


41it [1:23:33, 149.88s/it]

one line


42it [1:24:24, 120.08s/it]

one line


In [ ]:
1pred_ratings

,UserID,MovieID,Rating
0,6040.0,2.0,3.185331
1,6040.0,3.0,3.305444
2,6040.0,4.0,3.157618
3,6040.0,5.0,3.243234
4,6040.0,6.0,3.205683
...,...,...,...
9462,3966.0,3948.0,3.503990
9463,3966.0,3949.0,3.485174
9464,3966.0,3950.0,3.477988
9465,3966.0,3951.0,3.252062


In [ ]:
pred_ratings = pred_ratings.sort_values(by=['Rating'], ascending=False)

In [ ]:
for row in pred_ratings.head(10).itertuples():
  print(act_movies['title'].loc[act_movies['MovieID'] == row.MovieID])

In [ ]:
# this code tries to create a sparse matrix of user-movie and then fill in all movies ratings for each user
# from sklearn.metrics.pairwise import cosine_similarity

# # need to creat a user-movie matrix first
# # initiate all with zeroes 
# list_users = train['UserID'].unique()
# list_movies = act_movies['MovieID'].unique()
# user_movie = pd.DataFrame(0, list_users, list_movies)
# # print(user_movie)
# user_movie.at[6040, 1] = 1; # the first one is the index, which is again the UserID 

# user = 1
# # print(user_movie.iloc[0,1])
# # for user in user_movie.index: # this is the UserIDs 
# list_by_user = act_movies[['MovieID', 'encodings']].loc[act_movies['MovieID'].isin(act_ratings['MovieID'].loc[act_ratings['UserID'] == user])]
# # print(list_by_user['encodings'].iloc[0])

# import torch
# import numpy as np
# tmp = type(act_movies['encodings'].loc[act_movies['MovieID'] == 9])
# print(tmp)
# # print("======")
# # print(list_by_user.encodings.iloc[0])
# # print("======")
# # print(cosine_similarity(tmp, list_by_user.encodings.iloc[0]))
# # act_movies['encodings'][9]
# # print(cosine_similarity(np.array(tmp), list_by_user.encodings.iloc[0]))
# # for movie in user_movie.columns: 
# #   if ((movie in list_by_user['MovieID']) == False): 
# #     print(cosine_similarity(act_movies['encodings'].loc[act_movies['MovieID'] == movie], i) for i in list_by_user.encodings)
#     # [cosine_similarity(act_movies.at[0, 'encodings'], i) for i in test.encoding

Below is repeated code 

In [ ]:
# # act_ratings['MovieID'].loc[act_ratings['UserID'] == 1]
# # print(act_movies['encodings'].iloc[0] * 5)
# test = act_movies[['MovieID', 'encodings']].loc[act_movies['MovieID'].isin(act_ratings['MovieID'].loc[act_ratings['UserID'] == 1])]
# # print(test)
# # print(test.index[1])
# # print(act_ratings['Rating'].loc[(act_ratings['MovieID'] == 1) & (act_ratings['UserID'] == 1)])
# for movie in test.itertuples(): 
#   test.at[movie.Index, 'encodings'] = test.loc[movie.Index,'encodings'] * act_ratings['Rating'].loc[(act_ratings['MovieID'] == movie.MovieID) & (act_ratings['UserID'] == 1)]
#   # movie['encodings'] = movie.encodings * (act_ratings['Rating'].loc[(act_ratings['MovieID'] == movie.MovieID) & (act_ratings['UserID'] == 1)])
# print(test)

# # done for one user, need to iter all 

In [ ]:
# from sklearn.metrics.pairwise import cosine_similarity
# # def preprocess_text(user_id):
# #   text = 
# #   text = text.lower()
# #   text = re.sub('[^A-Za-z0-9]+', ' ', text)
# #   return text
  
# # query_text = preprocess_text()
# # query_encoding = get_bert_embeddings(query_text, preprocessor, encoder)

# # df_yt['similarity_score'] = df_yt['encodings'].apply(lambda x: metrics.pairwise.cosine_similarity(x, query_encoding)[0][0])
# # df_results = df_yt.sort_values(by=['similarity_score'], ascending=False)
# # query_encoding1 = get_bert_embeddings("test test test", preprocessor, encoder)
# # query_encoding2 = get_bert_embeddings("you", preprocessor, encoder)
# # print(cosine_similarity(query_encoding1, query_encoding2))

In [ ]:
# import torch
# import numpy as np
# from statistics import mean

# act_movies.at[0, 'encodings']
# # y_new_list = torch.mean(torch.stack([cosine_similarity(act_movies.at[0, 'encodings']) * i for i in test.encodings]))
# my_new_list = [cosine_similarity(act_movies.at[0, 'encodings'], i) for i in test.encodings]

In [ ]:
# np.array(my_new_list).mean()